## Knižnica keras

Knižnica `keras` poskytuje jednotné rozhranie pre vytváranie modelov neurónových sietí rôznych architektúr. Pri vytváraní a učení modelu budeme zvyčajne postupovať podľa nasledujúcich krokov:

1. Načítanie trénovacích a testovacích dát v podobe `numpy` polí.
2. Vytvorenie modelu neurónovej siete.
3. Voľba optimalizačnej metódy a príprava modelu na učenie (kompilácia).
4. Učenie modelu na trénovacích dátach.
5. Vyhodnotenie modelu na testovacích dátach.

Viac informácií o knižnici `keras` nájdete [tu](https://keras.io) (v angličtine).

### Načítanie dát

`keras` poskytuje príklady dátových množín pre základné typy úloh. V nasledujúcom príklade načítame dátovú množinu  [Boston Housing](https://www.kaggle.com/c/boston-housing), ktorá popisuje predaj nehnuteľností na predmestiach Bostonu. Dátová množina obsahuje 13 vstupných atribútov, ktoré popisujú napr. mieru kriminality, dopravné spojenie a služby v danej časti mesta. Cieľový atribút je číselný a popisuje priemernú cenu nehnuteľnosti.

In [ ]:
from tensorflow.keras.datasets import boston_housing

# načítame dáta z dátovej množiny do `numpy` polí, dáta sú priamo rozdelené na vstupné a výstupné atribúty
# a trénovaciu a testovaciu množinu, aby bolo možné zreplikovať rovnaké rozdelenie dát, je potrebné nastaviť
# inicializáciu náhodných čísel (`seed`)
(x_train, y_train), (x_test, y_test) = boston_housing.load_data(seed=1234)

Dáta majú tvar (počet trénovacích/testovacích príkladov x počet vstupných/výstupných atribútov).

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

### Vytvorenie modelu

Základný model doprednej neurónovej siete je možné vytvoriť sekvenčným prepojením jednotlivých vrstiev. Sekvenčný model je reprezentovaný objektom `Sequential`, do ktorého je možné pridať jednotlivé vrstvy. Základný typ vrstvy vytvára úplné prepojenie s predchádzajúcou vrstvou. V nasledujúcom príklade vytvoríme model s jednou skrytou vrstvou s 10 neurónmi a jednou výstupnou vrstvou s jedným výstupom pre cieľový atribút.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
# pre prvú vrstvu je potrebné zadať aj počet vstupov `input_dim`
# ako aktivačnú funkciu na skrytej vrstve definujeme hyperbolický tangens `tanh`
model.add(Dense(10, input_dim=13, activation='tanh'))
# pre regresiu nastavíme lineárnu výstupnú aktivačnú funkciu
# veľkosť priamo udáva počet výstupov (1)
model.add(Dense(1, activation='linear'))

Pomocou metódy `summary` je možné vypísať štruktúru modelu vrátane počtu parametrov.

In [ ]:
model.summary()

### Inicializácia váh

Váhy je možné inicializovať pomocou objektov z modulu `keras.initializers`. Pre základnú vrstvu doprednej siete je možné inicializovať samostatne váhy vstupov a váhy biasu.

In [ ]:
from tensorflow.keras import initializers

# váhy pre vstupy inicializujeme podľa Glorot inicializácie (nastavíme inicializáciu náhodného generátora `seed`)
# váhy pre bias (konštantný vstup s hodnotou 1) nastavíme na 0
layer = Dense(10,
              input_dim=13, kernel_initializer=initializers.glorot_normal(seed=1234),
              bias_initializer=initializers.Zeros())

# skrátene môžeme zadať aj reťazec s názvom inicializátora (v tomto prípade sa použijú prednastavené hodnoty
# argumentov inicializátora)
layer = Dense(10,
              input_dim=13, kernel_initializer="glorot_normal",
              bias_initializer="zeros")

### Konfigurácia optimalizačnej metódy

Knižnica `keras` definuje v module `keras.optimizers` rôzne typy optimalizačných metód. Typ `SGD` implementuje základnú metódu stochastického gradientového zostupu.

In [ ]:
from tensorflow.keras.optimizers import SGD

sgd = SGD(0.01) # parameter určuje rýchlosť učenia
# pred učením je potrebné model skompilovať pre danú optimalizačnú metódu
# pri kompilácii je potrebné vybrať optimalizovanú chybovú funkciu (pre regresiu priemernú kvadratický chybu
# `mean_squared_error`), voliteľne je možné zvoliť aj ďalšie metriky, ktoré sa vyhodnotia počas učenia na
# trénovacích dátach (napr. priemernú absolútnu chybu `mean_absolute_error`)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mean_absolute_error'])

Po skompilovaní je možné model naučiť metódou `fit`. Ako argumenty je potrebné určiť počet epoch (argument `epochs`) a počet príkladov, ktoré sa zahrnú do výpočtu gradientu pri každom kroku `batch_size`.

In [ ]:
f = model.fit(x_train, y_train, epochs=25, batch_size=10)

Priebeh učenia si môžeme vizualizovať pomocou knižnice `matplotlib`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(1)
plt.subplot(211)
plt.plot(f.history['loss'])
plt.subplot(212)
p = plt.plot(f.history['mean_absolute_error'])

### Testovanie modelu

Model je možné otestovať na testovacej množine príkladov pomocou metódy `evaluate`. Výsledok je pole, ktoré obsahuje hodnotu chybovej funkcie a všetkých metrík definovaných pri kompilácii modelu vyhodnotených na testovacej množine.

In [ ]:
scores = model.evaluate(x_test, y_test)
print('MSE: {:.4f}, MAE: {:.4f}'.format(scores[0], scores[1]))